In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import chromadb

chroma_client = chromadb.Client()

In [3]:
collection = chroma_client.create_collection(name='review_place_address_tags')

In [6]:
import preprocess_functions as pf

documents = pf.make_csv_to_documents_sep('찐찐찐찐막 전국 데이터.csv')

d:\SKN\3차 단위 프로젝트\rag관련\preprocess_functions.py:23: DtypeWarning: Columns (2,55,57,58,59,60,63,64,67,68,69,70,72) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv(file_path)


AttributeError: 'numpy.float64' object has no attribute 'replace'

In [ ]:
from langchain.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embedding_function = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L12-v2')
# 1. 컬렉션 생성
collection = chroma_client.create_collection(name='place_address_tags')

# 2. 벡터 스토어 객체 생성 (컬렉션 이름을 사용하여 연결)
vectorstore = Chroma(
    collection_name="place_address_tags",  # 컬렉션 이름
    embedding_function=embedding_function,  # 벡터화할 임베딩 함수
    persist_directory='./chroma_1'           # 벡터 DB를 저장할 디렉토리
)

In [ ]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
embedding_model = SentenceTransformer('all-MiniLM-L12-v2')


documents_to_store = []
for doc in documents:
    metadata = doc.metadata
    
    # place, 주소, 태그만 추출하여 새로운 metadata 만들기
    filtered_metadata = {
        "place": metadata.get("place", ""),
        "주소": metadata.get("주소", ""),
        "태그": metadata.get("태그", "")
    }
    
    # page_content로 청킹 수행
    page_content = f"{filtered_metadata['place']} {filtered_metadata['주소']} {filtered_metadata['태그']}"
    chunks = text_splitter.split_text(page_content)  # 청킹 수행

    # 각 청크에 대해 Document 객체로 감싸서 문서 목록에 추가
    for chunk in chunks:
        document = Document(
            page_content=chunk,  # 청크된 텍스트
            metadata=filtered_metadata  # metadata
        )
        documents_to_store.append(document)

# 임베딩 생성
embeddings = embedding_model.encode([doc.page_content for doc in documents_to_store])

# Chroma 벡터 DB에 문서 추가 (배치 처리)
batch_size = 5461  # Chroma에서 허용하는 최대 배치 크기

# 문서를 배치 단위로 나누어 추가
for i in range(0, len(documents_to_store), batch_size):
    batch_documents = documents_to_store[i:i + batch_size]
    batch_embeddings = embeddings[i:i + batch_size]
    
    # 벡터 DB에 배치 추가
    vectorstore.add_documents(
        documents=batch_documents,
        embeddings=batch_embeddings
    )

# 벡터 DB에 변경 사항 저장
vectorstore.persist()